## 2023638 | Anton Dementyev | Lab Assignment 2

In [276]:
from sklearn import datasets
import numpy as np
import math

In [277]:
iris = datasets.load_iris()

In [278]:
print(f'Dims: X - {iris.data.shape}, Y - {iris.target.shape}')
print('Samples:', iris.data[0], iris.target[0])
num_categories = len(np.unique(iris.target))
print(f'{num_categories} unique categories are present')
num_features = iris.data.shape[1]
print(f'{num_features} features are present in each sample')

Dims: X - (150, 4), Y - (150,)
Samples: [5.1 3.5 1.4 0.2] 0
3 unique categories are present
4 features are present in each sample


### Cross Validation

In [279]:
X_train, Y_train, X_test, Y_test = [], [], [], []
for i in range(len(iris.data)):
    if i % 40 >= 0 and i % 50 < 10:
        X_test.append(iris.data[i])
        Y_test.append(iris.target[i])
    else:
        X_train.append(iris.data[i])
        Y_train.append(iris.target[i])

### Likelihood

In [280]:
obj = {}
for i in range(num_categories):
    obj[i] = {'means': [], 'stds': []}
cat_length = len(X_train) / num_categories

for k in range(num_categories):
    slc = X_train[int(k * cat_length) : int((k+1) * cat_length)]
    for i in range(num_features):
        mean = sum([x[i] for x in slc]) / cat_length
        obj[k]['means'].append(mean)
    for i in range(num_features):
        mean = obj[k]['means'][i]
        obj[k]['stds'].append(math.sqrt(sum([(x[i] - mean) ** 2 for x in slc]) / cat_length))
    
print(obj)

{0: {'means': [5.0425, 3.4574999999999996, 1.4649999999999994, 0.2525], 'stds': [0.35558929961403507, 0.38787079034131977, 0.18513508581573615, 0.10951598056904757]}, 1: {'means': [5.894999999999999, 2.7449999999999997, 4.2325, 1.3124999999999998], 'stds': [0.4460661385938188, 0.3024483426967323, 0.46172908723622785, 0.20147890708458788]}, 2: {'means': [6.592500000000001, 2.982499999999999, 5.497499999999999, 2.0224999999999995], 'stds': [0.5913490931759346, 0.3185023547793643, 0.5265394097311237, 0.27063582541858716]}}


In [281]:
# calculating probability density
def likelihood(x, c):
    mult = 1
    for i in range(len(x)):
        mean = obj[c]['means'][i]
        std = obj[c]['stds'][i]
        lh = 1 / (math.sqrt(2 * math.pi) * std) * math.exp(-0.5*(((x[i] - mean) ** 2) / (std ** 2)))
        mult = mult * lh
    return mult

### Priori

In [282]:
# Since all categories are equal, the category priori probability is the same
priori_c = 1 / num_categories

### Posterior

In [283]:
priori_x = 1 / len(X_train)

In [284]:
def posterior(x):
    mx = (0, -1)
    for i in range(num_categories):
        inter = likelihood(x, i) * priori_c
        if inter > mx[0]:
            mx = (inter, i)
    return (mx[0] / priori_x, mx[1])

### Performance Evaluation

In [285]:
def evaluate(X_test, Y_test):
    correct = 0
    for x, y in zip(X_test, Y_test):
        if posterior(x)[1] == y:
            correct = correct + 1
    return correct / len(X_test)

In [286]:
result = evaluate(X_test, Y_test)
print(f'The classifier has reached {result*100}% on the testing set')

The classifier has reached 93.33333333333333% on the testing set


### Challenging Task